# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298425940305                   -0.85    5.1
  2   -8.300193706296       -2.75       -1.26    1.2
  3   -8.300446839880       -3.60       -1.89    3.4
  4   -8.300462077012       -4.82       -2.75    1.1
  5   -8.300464292461       -5.65       -3.04    5.2
  6   -8.300464442058       -6.83       -3.20    1.0
  7   -8.300464537089       -7.02       -3.34    6.6
  8   -8.300464587196       -7.30       -3.46    1.9
  9   -8.300464629757       -7.37       -3.67    1.0
 10   -8.300464633442       -8.43       -3.73    1.4
 11   -8.300464640606       -8.14       -3.91    1.0
 12   -8.300464643957       -8.47       -4.47    1.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63205002646                   -0.70    5.4
  2   -16.63629620237       -2.37       -1.15    1.0
  3   -16.67921162256       -1.37       -1.72    3.9
  4   -16.67928364968       -4.14       -2.21    5.5
  5   -16.67928560474       -5.71       -2.91    1.4
  6   -16.67928615071       -6.26       -3.24    4.2
  7   -16.67928619349       -7.37       -3.37    1.8
  8   -16.67928622286       -7.53       -3.99    1.0
  9   -16.67928622425       -8.86       -4.42    2.1


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32685431595                   -0.56    7.4
  2   -33.33442631795       -2.12       -1.00    1.1
  3   -33.33600407543       -2.80       -1.73    7.8
  4   -33.33685457324       -3.07       -2.33    4.6
  5   -33.33693866361       -4.08       -2.93    8.1
  6   -33.33694388322       -5.28       -3.79    3.0
  7   -33.33694391867       -7.45       -4.19    5.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298230882010                   -0.85    5.1
  2   -8.300226940090       -2.70       -1.59    1.0
  3   -8.300455760505       -3.64       -2.61    5.0
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       3
│       3
│       3
│       3
│       2
│       5
│       2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  4   -8.300407467448   +   -4.32       -2.38   15.1
  5   -8.300464457044       -4.24       -3.63    2.2
  6   -8.300464594931       -6.86       -3.96    2.8
  7   -8.300464643588       -7.31       -4.94    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32528151430                   -0.56    6.6
  2   -33.32558769387       -3.51       -1.27    1.0
  3   -22.88727192667   +    1.02       -0.56    8.8
  4   -33.27043305782        1.02       -1.59    7.1
  5   -33.14349799880   +   -0.90       -1.27    3.9
  6   -32.04687012352   +    0.04       -1.01    4.4
  7   -33.32838553236        0.11       -1.95    4.2
  8   -33.33366267102       -2.28       -2.14    2.5
  9   -33.33578777193       -2.67       -2.26    2.5
 10   -33.33625456356       -3.33       -2.60    1.9
 11   -33.33681474610       -3.25       -2.96    2.2
 12   -33.33692156970       -3.97       -3.32    3.8
 13   -33.33694089992       -4.71       -3.64    3.2
 14   -33.33694355722       -5.58       -3.92    3.0
 15   -33.33694294365   +   -6.21       -4.02    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.